**Chatbot con búsqueda en lenguaje natural**

Ahora integraremos la capacidad de búsqueda en lenguaje natural en el chatbot. El comportamiento esperado es que, cuando el usuario final envía una solicitud de búsqueda, recuperamos de una base de conocimiento una o más entradas que son más relevantes para el texto de la consulta y se las devolvemos al usuario final como resultado de la búsqueda.

**Knowledge base**

Una base de conocimiento es una base de datos que contiene información sobre un producto, un servicio, un tema o un dominio de conocimiento. Para habilitar la búsqueda en lenguaje natural en la base de conocimiento, debemos convertir cada entrada de datos de la base de conocimiento en incrustación y almacenarla para la búsqueda. Con un texto de consulta determinado, convertimos el texto de consulta en su incrustación y buscamos las entradas de datos más cercanas al texto de consulta.

Se tarda unos 3 minutos en generar la incrustación de 1000 frases de forma secuencial.

In [2]:
region = 'us-west-2'
host = 'https://5tl3o6l27r1pcm2hddk9.us-west-2.aoss.amazonaws.com'

In [3]:
import boto3
import json
import requests
from requests_aws4auth import AWS4Auth

def get_embedding(bedrock, text):
    modelId = 'amazon.titan-embed-text-v1'
    accept = 'application/json'
    contentType = 'application/json'
    input = {
            'inputText': text
        }
    body=json.dumps(input)
    response = bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept,contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body['embedding']
    return embedding
    
# main function
service = 'aoss'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)
index = 'demo-index'
datatype = '_doc'
url = host + '/' + index + '/' + datatype
headers = {'Content-Type': 'application/json'}
bedrock = boto3.client(service_name='bedrock-runtime')
dataset = {
    'The theory of general relativity says that the observed gravitational effect between masses results from their warping of spacetime.',
    'Quantum mechanics allows the calculation of properties and behaviour of physical systems. It is typically applied to microscopic systems: molecules, atoms and sub-atomic particles.', 
    'Wavelet theory is essentially the continuous-time theory that corresponds to dyadic subband transforms — i.e., those where the L (LL) subband is recursively split over and over.',
    'Every particle attracts every other particle in the universe with a force that is proportional to the product of their masses and inversely proportional to the square of the distance between their centers.',
    'The electromagnetic spectrum is the range of frequencies (the spectrum) of electromagnetic radiation and their respective wavelengths and photon energies.'
}
for entry in dataset:
    embedding = get_embedding(bedrock, entry)
    document = {
        'embedding': embedding,
        'content': entry
    }
    response = requests.post(url, auth=awsauth, json=document, headers=headers)


**Realizar una búsqueda**

En el siguiente código de ejemplo se muestra cómo realizar una búsqueda entre el conjunto de datos. El texto de la consulta es "Albert Einstein" y queremos que se devuelvan 3 entradas.

In [4]:
import boto3
import requests
from requests_aws4auth import AWS4Auth

def get_embedding(bedrock, text):
    modelId = 'amazon.titan-embed-text-v1'
    accept = 'application/json'
    contentType = 'application/json'
    input = {
            'inputText': text
        }
    body=json.dumps(input)
    response = bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept,contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body['embedding']
    return embedding

def search(region, host, index, embedding, limit):
    credentials = boto3.Session().get_credentials()
    awsauth = AWS4Auth(
        credentials.access_key, 
        credentials.secret_key, 
        region, 
        "aoss", 
        session_token=credentials.token
    )
    datatype = '_search'
    url = host + '/' + index + '/' + datatype
    headers = {'Content-Type': 'application/json'}
    document = {
        'size': limit,
        'query': {
            'knn': {
                'embedding': {
                    'vector': embedding,
                    'k': limit
                }
            }
        }
    }
    response = requests.get(url, auth=awsauth, json=document, headers=headers)
    response.raise_for_status()
    return response.json()

# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)
query = 'Albert Einstein'
embedding = get_embedding(bedrock, query)
index = 'demo-index'
limit = 3
result = search(region, host, index, embedding, limit)

for item in result['hits']['hits']:
    print(item['_source']['content'])


Albert Einstein teaches physics in London.
Albert Einstein teaches physics in Paris.
Albert Einstein works as a support engineer in London.


**Integración de chatbot**

Ahora modificaremos el código de nuestra función Lambda para agregar la capacidad de realizar búsquedas en lenguaje natural. Si el símbolo del sistema humano comienza con, se trata como un comando de búsqueda. El chatbot extrae el contenido después del comando de búsqueda como texto de consulta, busca N entradas más relevantes de la base de conocimientos y las devuelve al usuario final. //search

**Código de función de Lambda**

Navegue a la carpeta **Chatbot**, use el menú **Archivo -> Nuevo -> Archivo de Python** para crear un nuevo archivo de Python. Esto crea un archivo de Python sin título en la carpeta **Chatbot**. Cambie el nombre del archivo a **chatbot_v2.py**. Copie y pegue el siguiente contenido en **chatbot_v2.py**. Utilice el menú **Archivo -> Guardar archivo de Python** para guardar el contenido del archivo.

En este código, obtenemos el punto de conexión DNS de la colección OpenSearch Serverless de una variable de entorno aossHost en la función Lambda. Configuramos esta variable de entorno cuando creamos la función de Lambda en CloudFormation.

In [ ]:
import json
import boto3
import os
import requests
from requests_aws4auth import AWS4Auth

region = os.environ.get('AWS_REGION')
bedrock = boto3.client(service_name='bedrock-runtime')
aoss_host = os.environ.get('aossHost')

def lambda_handler(event, context):
    if (event['httpMethod'] == 'GET'):
        output = load_html()
        return {
            'statusCode': 200,
            'headers': {'Content-Type': 'text/html'},
            'body': output
        }
    elif (event['httpMethod'] == 'POST'):
        prompt = event['body']
        comm, query = check_prompt_command(prompt)
        if comm == 'search':
            output = search(query, limit=3)
        else:
            output = chat(prompt)
        return {
            'statusCode': 200,
            'headers': {'Content-Type': 'text/html'},
            'body': output
        }
    else:
         return {
            'statusCode': 200,
            'headers': {'Content-Type': 'text/html'},
            'body': "OK"
        }

def check_prompt_command(prompt):
    comm = 'chat'
    query = None
    # Check the last line of the prompt to see if it is a search request.
    lines = prompt.splitlines()
    last_line = lines[-1]
    # Check if the last line starts with "Human: "
    if last_line.startswith('Human: '):
        last_line = last_line[7:].strip()
        # Check if the human prompt starts with "//search "
        if last_line.startswith('//search '):
            query = last_line[9:].strip()
            if query != None:
                comm = 'search'
    return comm, query

def load_html():
    html = ''
    with open('index.html', 'r') as file:
        html = file.read()
    return html

def chat(prompt):
    modelId = 'ai21.j2-ultra'
    accept = 'application/json'
    contentType = 'application/json'
    body=json.dumps({'prompt': prompt, 'maxTokens': 250, 'stopSequences': ['Human:']})
    response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept,contentType=contentType)
    response_body = json.loads(response.get('body').read())
    completions = response_body['completions']
    output = ''
    for part in completions:
        output += part['data']['text']
    return output

def search(query, limit=1):
    # get embedding
    embedding = get_embedding(query)
    # prepare for OpenSearch Serverless
    service = 'aoss'
    credentials = boto3.Session().get_credentials()
    awsauth = AWS4Auth(
        credentials.access_key, 
        credentials.secret_key, 
        region, 
        service, 
        session_token=credentials.token
    )
    # search
    index = 'demo-index'
    datatype = '_search'
    url = aoss_host + '/' + index + '/' + datatype
    headers = {'Content-Type': 'application/json'}
    document = {
        'size': limit,
        'query': {
            'knn': {
                'embedding': {
                    'vector': embedding,
                    'k': limit
                }
            }
        }
    }
    # response
    response = requests.get(url, auth=awsauth, json=document, headers=headers)
    response.raise_for_status()
    data = response.json()
    output = ''
    for item in data['hits']['hits']:
        output += item['_source']['content'] + '\n'
    return output.strip()

def get_embedding(text):
    modelId = 'amazon.titan-embed-text-v1'
    accept = 'application/json'
    contentType = 'application/json'
    input = {
            'inputText': text
        }
    body=json.dumps(input)
    response = bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept,contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body['embedding']
    return embedding


La lógica del código es:

* Para todas las solicitudes GET, cargamos el contenido HTML de index.html y se lo devolvemos al solicitante.
* Para todas las solicitudes POST, extraemos el cuerpo de la solicitud y, a continuación,
* Si la última línea de human comienza con , esto se trata como un comando de búsqueda, cualquier cosa después del comando de búsqueda se trata como el texto de la consulta. En este caso, convertimos el texto de la consulta en incrustación y, a continuación, buscamos en la base de conocimiento. Las 3 entradas de datos más cercanas en la base de conocimientos se devuelven al solicitante.//search
* De lo contrario, usamos el cuerpo de la solicitud como solicitud para invocar un modelo básico y, a continuación, devolvemos la respuesta del modelo básico al solicitante.
* Para todas las demás solicitudes, simplemente devolvemos un OK al solicitante.

**Paquete de implementación**
En la ventana del terminal, vaya a la carpeta Chatbot:

In [ ]:
En la terminal ejecutar : 

cd ~/Chatbot

pip install --target ./package requests --no-deps
pip install --target ./package requests_aws4auth --no-deps
pip install --target ./package charset_normalizer --no-deps
pip install --target ./package idna --no-deps
pip install --target ./package certifi --no-deps

Ejecute los siguientes comandos para volver a compilar el paquete de implementación:

rm my_deployment_package.zip
cd package
zip -r ../my_deployment_package.zip .
cd ..
zip my_deployment_package.zip index.html
zip my_deployment_package.zip chatbot_v1.py
zip my_deployment_package.zip chatbot_v2.py

Copie el archivo zip en su bucket de S3.

S3_BUCKET=$(aws s3 ls | grep bedrock-workshop | cut -d' ' -f3-)
aws s3 cp my_deployment_package.zip s3://$S3_BUCKET

Despliegue
Ahora tenemos que configurar la función de Lambda para utilizar el nuevo paquete de implementación:

aws lambda update-function-code --function-name BedrockWorkshopChatbot --s3-bucket $S3_BUCKET --s3-key my_deployment_package.zip

También necesitamos configurar la función Lambda para usar el controlador en chatbot_v2.py.

aws lambda update-function-configuration --function-name BedrockWorkshopChatbot --handler chatbot_v2.lambda_handler

Acceda nuevamente a la interfaz de usuario web de su chatbot, pruebe con las siguientes indicaciones una por una para ver cómo funcionan las cosas (haga una pregunta a la vez). Si el mensaje del usuario final comienza con , el chatbot realizará una búsqueda en la base de conocimiento y devolverá la entrada de datos más cercana de la base de conocimiento.//search
Comienza con una charla informal:

Si recibe el siguiente mensaje de error del chatbot, debe revisar los registros de CloudWatch generados por la función de Lambda para ver qué está pasando.